# Data analysis of Bind-n-seq using Bicc1 protein


The first version was written in Julia 1.1.1.
The current version is written in Julia 1.6.0. (2021-05-05)

Written by Kaoru R Komatsu and Emi Miyashita  
Conducted by Kaoru R Komatsu

Quality filters remove the following reads.
+ Duplicated reads
+ Average Qscore is less than 30.
+ Length is not 20nt.

In [10]:
# Using BioJulia for counting K-mer
using Bio.Seq
#comp = composition(each(DNAKmer{4}, dna"ACGTTTCGAACGT"))#test run

In [11]:
using DataFrames,Statistics,StatsBase, Gadfly, CSV

In [12]:
function generate_mers(alphabet, k)
    kmer_set = [""]
    for i in 1:k
        new_kmers =[o * alphabet[j] for o in kmer_set for j in 1:length(alphabet)]
        kmer_set = new_kmers
    end
    sort(kmer_set)
    return kmer_set
end
                
    
function generate_mers_rank_map(alphabet, k)

    kmer_set = generate_mers(alphabet, k)
    rank_dict = Dict()
    for (i, o) in enumerate(kmer_set)
        rank_dict[o] = 0
    end
    return rank_dict
end
                
function str2kmers(s, k)
    out = []
    for i in 1:length(s)- k +1
        push!(out,s[i:i+k-1])
    end
    return out
end
                
                
function Topkmer(kmer_set,rank_map,datasets)
    out =  ist()
    for kmer in kmer_set
        ten = list()
        rank = rank_map[kmer]  
        for d in sorted(datasets)
            push!(ten, counts_by_name[d][rank])#Dict->list->num
        end
        push!(out,sum(ten))
    end
    top_enriched_kmer_list = [[i,v/sum(out)] for (i,v) in zip(kmer_set,out) if v == max(out)]
    return(top_enriched_kmer_list[0][0])
end

Topkmer (generic function with 1 method)

In [13]:
function open_fastq_and_profile_kmer(fastq_file, knum, qscore)
    dna_alphabet = ["A", "C", "G", "T"]
    kmer_set = generate_mers(dna_alphabet, knum)
    rank_map = generate_mers_rank_map(dna_alphabet, knum)

    reader = open(FASTQ.Reader, fastq_file)
    linecount = 0
    linecount_highqc = 0
    sequencedict = Dict{String,Int64}()
    duplicated_dict = Dict{String,Int64}()
    
    println("#######Loading#######")
    duplicate = false
    for record in reader 
        tseq = (convert(String,FASTQ.sequence(record)))
        qc = FASTQ.quality(record)
        linecount +=1
    
        
        if (length(tseq) == 20) && (mean(qc) >= qscore)
            
            # To remove PCR duplicates
            if haskey(sequencedict,tseq)
                duplicate = true
                if haskey(duplicated_dict,tseq)
                    duplicated_dict[tseq]  += 1
                else
                     duplicated_dict[tseq]  = 1
                end
            else
                duplicate = false
            end
                
            linecount_highqc +=1
            comp = composition(each(DNAKmer{knum},DNASequence(string(FASTQ.sequence(record)))))
            
            for (kmer, count) in convert(Dict,comp)
                
                if (!occursin("N", string(kmer))) 
                    if (duplicate ==false)
                        prevcount = rank_map[convert(String,kmer)]
                        rank_map[convert(String,kmer)] = prevcount + count
                        sequencedict[tseq] = 1
                    end
                end
            end
            
        end
        
    end
    println("total ",linecount," sequences")
    println("High QC ",linecount_highqc," sequences")
    println("(Removed) Suspected PCR-replicates: ", length(duplicated_dict))
    println("Removed ", length(duplicated_dict),"reads")
    return rank_map
    close(reader)
end


function df2kmer_table(file1,file2,outputname)

    dfL2 = sort(DataFrame(Dict(reverse(collect(file1)))))
    dfL4 = sort(DataFrame(Dict(reverse(collect(file2)))))
    Bicc1 = collect(dfL2[1,:])/sum(collect(dfL2[1,:]))
    Control = collect(dfL4[1,:])/sum(collect(dfL4[1,:]))
    Enrich_Div = Bicc1./Control 
    zEnrich_Div = zscore(Enrich_Div)
    Zscore = zscore(collect(map((x,y)-> (x ./sum(dfL2[1,:]))/(y ./sum(dfL4[1,:])), values(dfL2[1,:]), values(dfL4[1,:]))))

    dfL24 = DataFrame(
        Kmer=names(dfL2),
        Frequency_Bicc1 = collect(dfL2[1,:]),
        Frequency_Control = collect(dfL4[1,:]),
        NormalizedFrequency_Bicc1 = Bicc1 ,
        NormalizedFrequency_Control = Control,
        RelativeFrequency = Enrich_Div,
        RelativeFrequency_Zscore = zEnrich_Div)
            
    sort!(dfL24, :RelativeFrequency, rev=true)
    dfL24 |> CSV.write(outputname ,delim=',',writeheader=true)        
    return dfL24
    
end

df2kmer_table (generic function with 1 method)

# Analysis

In [14]:
cd("/Users/krk/Desktop/Bicc1-dand5/Rawdata/")
for i in [4,5,6]
    kmer_Bicc = open_fastq_and_profile_kmer("./190523_bindN/Data/Intensities/BaseCalls/L3_RBNS_BICCd4.fastq",i, 30)
    kmer_Mock = open_fastq_and_profile_kmer("./190523_bindN/Data/Intensities/BaseCalls/L4_Mock.fastq",i, 30)
    dfout = df2kmer_table(kmer_Bicc, kmer_Mock,"RelativeFrequency_$(i)mer.csv")
end

#######Loading#######
total 832224 sequences
High QC 715319 sequences
(Removed) Suspected PCR-replicates: 183
Removed 183reads
#######Loading#######
total 937217 sequences
High QC 780761 sequences
(Removed) Suspected PCR-replicates: 8757
Removed 8757reads
#######Loading#######
total 832224 sequences
High QC 715319 sequences
(Removed) Suspected PCR-replicates: 183
Removed 183reads
#######Loading#######
total 937217 sequences
High QC 780761 sequences
(Removed) Suspected PCR-replicates: 8757
Removed 8757reads
#######Loading#######
total 832224 sequences
High QC 715319 sequences
(Removed) Suspected PCR-replicates: 183
Removed 183reads
#######Loading#######
total 937217 sequences
High QC 780761 sequences
(Removed) Suspected PCR-replicates: 8757
Removed 8757reads
